In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
styles = ['隸', '楷', '篆', '行', '草']
column_names = ["key", "character", "style", "author", "image"]
image_size = 28

In [14]:
dataframe = pd.read_csv("/tf/hanzi/data.csv", header=None, names = column_names)
authors = dataframe["author"].unique()
dataset_size = dataframe.shape[0]
dataframe

,key,character,style,author,image
0,宀,容,隸,何绍基,宀/容/png/隸/何绍基-87773-256x256.png
1,宀,容,隸,马王堆帛书,宀/容/png/隸/马王堆帛书-87777-256x256.png
2,宀,容,隸,辟雍碑,宀/容/png/隸/辟雍碑-87771-256x256.png
3,宀,容,隸,马王堆帛书,宀/容/png/隸/马王堆帛书-87779-256x256.png
4,宀,容,隸,居延简,宀/容/png/隸/居延简-87774-256x256.png
...,...,...,...,...,...
166666,辰,辰,草,陈道复,辰/辰/png/草/陈道复-27196-256x256.png
166667,辰,辰,草,徐伯清,辰/辰/png/草/徐伯清-27208-256x256.png
166668,辰,辰,草,沈粲,辰/辰/png/草/沈粲-27202-256x256.png
166669,辰,辰,草,怀素,辰/辰/png/草/怀素-27197-256x256.png


In [20]:
def get_processed_dataset(fileList, positions):
    dataset = get_raw_dataset(fileList, positions)
    return process_dataset(dataset)

def process_dataset(dataset):
    dataset = dataset.batch(32)
    dataset = dataset.repeat(10)
    return dataset.map(lambda x: (x["image"], x["style"]))

def get_raw_dataset(fileList, positions):
    files = tf.data.Dataset.from_tensor_slices(fileList)
    dataset = files.interleave(tf.data.TextLineDataset, 1)
    return dataset.skip(positions[0]).take(positions[1]).map(parser(), num_parallel_calls=8)

def parser():
    styles_mapping = tf.constant(styles)
    styles_table = tf.contrib.lookup.index_table_from_tensor(mapping=styles_mapping, default_value=-1)
    
    authors_mapping = tf.convert_to_tensor(authors)
    authors_table = tf.contrib.lookup.index_table_from_tensor(mapping=authors_mapping, default_value=-1)
    
    def parse(value):
        nonlocal styles_table, authors_table
        columns_default = [[""], [""], [""], [""], [""]]
        columns = tf.decode_csv(value, record_defaults=columns_default, field_delim=',')
        features = dict(zip(column_names, columns))
        for f, tensor in features.items():
            if f == "style":
                features["style"] = styles_table.lookup(tensor)
            if f == "author":
                features["author"] = authors_table.lookup(tensor)
            if f == "image":
                image_string = tf.read_file(tf.strings.join([tf.constant("/tf/hanzi"), tensor], '/'))
                image_decoded = tf.image.decode_png(image_string, channels=1)
                image_resized = tf.image.resize_images(image_decoded, [image_size, image_size])
                # image_standardized = tf.image.per_image_standardization(image_resized)
                image_reshaped = tf.reshape(image_resized, [image_size, image_size])
                features["image"] = image_reshaped
        return features
    return parse

def get_train_dataset():
    return get_processed_dataset(["/tf/hanzi/shuffled_data.csv"], (0, 120000))

def get_validation_dataset():
    return get_processed_dataset(["/tf/hanzi/shuffled_data.csv"], (120000, 20000))

def get_test_dataset():
    return get_processed_dataset(["/tf/hanzi/shuffled_data.csv"], (140000, dataset_size - 140000))

train_dataset = get_train_dataset()

iterator = train_dataset.make_initializable_iterator()
el = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    tf.tables_initializer().run()
    plt.figure(figsize=(image_size, image_size))
    record = sess.run(el)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(record[0][i], cmap=plt.cm.binary)
    plt.show()

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [48]:
def cnn_model_fn(features, labels, mode):
  one_hot_style = tf.one_hot(labels, 5)

  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features, [-1, image_size, image_size, 1], name = "ReshapeInput")

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=5)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.softmax_cross_entropy(one_hot_style, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [51]:
style_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn
)

tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

style_classifier.train(
    input_fn=get_train_dataset,
    steps=10000,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_save_summary_steps': 100, '_protocol': None, '_num_ps_replicas': 0, '_device_fn': None, '_log_step_count_steps': 100, '_is_chief': True, '_task_type': 'worker', '_tf_random_seed': None, '_model_dir': '/tmp/tmpybklolrl', '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f469025d588>, '_save_checkpoints_secs': 600, '_task_id': 0, '_eval_distribute': None}
INFO:tensorflow:Calling model_fn.
called
called
called
cnn_model_fn
Tensor("IteratorGetNext:0", shape=(?, 28, 28), dtype=float32)
Tensor("Ite

INFO:tensorflow:global_step/sec: 14.9125
INFO:tensorflow:probabilities = [[0.14754051 0.15687771 0.15317436 0.21480532 0.32760206]
 [0.20417085 0.19192328 0.18747531 0.17579068 0.24063988]
 [0.19320443 0.18720526 0.19014612 0.22069457 0.20874956]
 [0.06316014 0.08587721 0.11855593 0.36312783 0.36927888]
 [0.1956661  0.18436    0.18807037 0.21758522 0.2143183 ]
 [0.19228345 0.18438853 0.18669194 0.22411044 0.21252567]
 [0.0842099  0.04913034 0.04504299 0.37707284 0.444544  ]
 [0.23589733 0.05763238 0.10869285 0.29739243 0.30038497]
 [0.12512921 0.19266447 0.11876824 0.1979713  0.36546677]
 [0.20657344 0.15661395 0.19051374 0.23168933 0.21460949]
 [0.19263043 0.1864874  0.19081958 0.22144493 0.20861763]
 [0.19209503 0.18634929 0.18809776 0.22158505 0.21187286]
 [0.19056009 0.18700017 0.18350022 0.22042596 0.21851356]
 [0.19430639 0.18581566 0.18941699 0.22195676 0.2085042 ]
 [0.10919642 0.17394868 0.15216114 0.24539442 0.31929937]
 [0.03517239 0.01636713 0.05335622 0.48098844 0.41411588]

INFO:tensorflow:loss = 1.4585292, step = 401 (6.186 sec)
INFO:tensorflow:probabilities = [[0.18385564 0.18346198 0.17976579 0.23683728 0.21607935]
 [0.19291306 0.18164845 0.17116587 0.24413437 0.21013825]
 [0.18707982 0.18202442 0.17993629 0.23898566 0.2119738 ]
 [0.01284888 0.0027053  0.00611809 0.83424807 0.14407963]
 [0.08916991 0.03051995 0.02811572 0.6167306  0.23546389]
 [0.09795146 0.04958488 0.04730349 0.42922643 0.37593374]
 [0.1842105  0.17777498 0.17795144 0.23451065 0.22555242]
 [0.16271019 0.12028666 0.06358647 0.29425386 0.3591628 ]
 [0.18813902 0.18341026 0.17676385 0.2359844  0.21570253]
 [0.12092876 0.10401168 0.06621795 0.33312517 0.3757164 ]
 [0.19291143 0.12781385 0.12613109 0.2287198  0.3244239 ]
 [0.15965931 0.18370329 0.14396916 0.21576673 0.2969016 ]
 [0.18337017 0.1852144  0.13982087 0.23845786 0.25313663]
 [0.15064532 0.1605777  0.12773615 0.33041817 0.23062275]
 [0.16334696 0.1329229  0.060103   0.24392635 0.39970076]
 [0.14233951 0.09628008 0.10083685 0.3598

INFO:tensorflow:global_step/sec: 15.7948
INFO:tensorflow:probabilities = [[0.18564472 0.18373321 0.17628624 0.2386857  0.21565011]
 [0.06341527 0.06856566 0.0282746  0.44473696 0.39500755]
 [0.09137053 0.09378068 0.05354867 0.41860473 0.34269533]
 [0.05694363 0.06219215 0.02212941 0.18045317 0.6782816 ]
 [0.1871666  0.18503043 0.16996954 0.24131632 0.21651708]
 [0.18491012 0.18330288 0.17452168 0.24092357 0.2163417 ]
 [0.1890789  0.18240209 0.17313845 0.2400931  0.21528746]
 [0.02539973 0.02713905 0.00999533 0.32568002 0.6117859 ]
 [0.14789456 0.13553949 0.0940411  0.35122868 0.2712962 ]
 [0.02304445 0.02165884 0.00786414 0.642032   0.30540055]
 [0.1859091  0.18034409 0.17196342 0.24496818 0.21681516]
 [0.1724477  0.16201627 0.11974016 0.28506392 0.26073194]
 [0.18733004 0.18664455 0.17271145 0.24002981 0.21328415]
 [0.03425912 0.02041034 0.01065798 0.4038476  0.53082496]
 [0.09040532 0.13878489 0.07992008 0.36320755 0.3276822 ]
 [0.09686921 0.10540877 0.07452098 0.37185228 0.35134882]

INFO:tensorflow:loss = 1.1861906, step = 901 (7.192 sec)
INFO:tensorflow:probabilities = [[0.1832845  0.18122208 0.16494355 0.25030324 0.22024661]
 [0.15986553 0.17398532 0.12135059 0.2854761  0.25932246]
 [0.15525274 0.15505208 0.083589   0.24036512 0.3657411 ]
 [0.16494532 0.15453224 0.14347918 0.2642275  0.27281573]
 [0.03655336 0.03496895 0.03517179 0.3425138  0.55079204]
 [0.05866408 0.05134149 0.02565865 0.55836326 0.30597258]
 [0.11755977 0.14889944 0.03296645 0.15197727 0.5485971 ]
 [0.12928876 0.16607441 0.08172114 0.24003714 0.3828786 ]
 [0.13726525 0.10762707 0.08132362 0.39556405 0.27822   ]
 [0.18140012 0.18407583 0.16515537 0.25201258 0.21735603]
 [0.15964872 0.15537114 0.15844174 0.2576073  0.268931  ]
 [0.1612238  0.18599877 0.11122673 0.18784918 0.35370147]
 [0.03136212 0.03764841 0.01788741 0.56582624 0.34727585]
 [0.1685027  0.08564325 0.06945683 0.32014063 0.3562566 ]
 [0.17480572 0.18085618 0.06694072 0.24389778 0.33349958]
 [0.1253386  0.10291316 0.05981905 0.2826

...

INFO:tensorflow:loss = 0.7449689, step = 8901 (6.145 sec)
INFO:tensorflow:probabilities = [[0.00008109 0.00015056 0.00031548 0.03948629 0.95996666]
 [0.00006933 0.00031415 0.00016073 0.07426143 0.9251944 ]
 [0.00304171 0.00619201 0.00112271 0.33096737 0.65867615]
 [0.02423785 0.11004184 0.01639024 0.24844828 0.6008819 ]
 [0.02476303 0.04458777 0.00038446 0.4737173  0.4565474 ]
 [0.01100474 0.19651844 0.06521924 0.39236057 0.33489695]
 [0.03989836 0.62288034 0.0496865  0.2179628  0.06957196]
 [0.00013963 0.00184507 0.00005116 0.12817349 0.8697906 ]
 [0.00056195 0.05330982 0.00062276 0.41396222 0.5315433 ]
 [0.00002683 0.00019604 0.00047463 0.13352276 0.8657797 ]
 [0.04676175 0.8263658  0.11489599 0.01011932 0.00185712]
 [0.00012026 0.01177279 0.00301444 0.09481669 0.89027584]
 [0.00166334 0.13706225 0.00158147 0.44483003 0.41486293]
 [0.02181669 0.20874839 0.02696129 0.3406076  0.40186596]
 [0.00000172 0.00020834 0.00015536 0.08045346 0.91918117]
 [0.04683116 0.09733012 0.00112465 0.628

INFO:tensorflow:global_step/sec: 14.9967
INFO:tensorflow:probabilities = [[0.00216345 0.0037498  0.00283285 0.81845146 0.1728024 ]
 [0.00395772 0.0150844  0.00048749 0.81266516 0.16780527]
 [0.9980494  0.00108069 0.00086918 0.00000088 0.00000001]
 [0.03502621 0.0283866  0.92495507 0.01088752 0.0007446 ]
 [0.4019954  0.05850468 0.00974713 0.3061552  0.22359766]
 [0.17797625 0.32573378 0.00516225 0.34964216 0.14148542]
 [0.00055547 0.01513321 0.08068311 0.7324874  0.17114088]
 [0.00283718 0.17539428 0.01018656 0.615435   0.19614692]
 [0.01116078 0.0953051  0.00021539 0.7892474  0.10407137]
 [0.12060286 0.6775816  0.0625552  0.12737806 0.01188227]
 [0.0000131  0.00001463 0.00000197 0.28920242 0.7107679 ]
 [0.00139534 0.01356301 0.00324427 0.30683234 0.674965  ]
 [0.02151564 0.1515456  0.03539947 0.69099396 0.10054526]
 [0.00002689 0.00069738 0.00099741 0.26370105 0.7345773 ]
 [0.02489213 0.07808502 0.10177036 0.5333752  0.26187733]
 [0.27673575 0.58526397 0.00062756 0.13571852 0.00165419]

INFO:tensorflow:loss = 0.8316697, step = 9401 (6.106 sec)
INFO:tensorflow:probabilities = [[0.00019338 0.00987255 0.00011864 0.35594872 0.6338667 ]
 [0.3756975  0.23850167 0.01149929 0.27243197 0.1018696 ]
 [0.0152901  0.01524277 0.06695939 0.54620624 0.35630155]
 [0.03189051 0.0307453  0.0001908  0.6025863  0.33458698]
 [0.03081332 0.05234063 0.9166195  0.00019315 0.00003349]
 [0.00236707 0.00495192 0.00017512 0.30106932 0.6914366 ]
 [0.99768233 0.00039826 0.0019059  0.00000868 0.00000475]
 [0.98560286 0.01179928 0.00092421 0.00164703 0.00002666]
 [0.01806633 0.14379828 0.01473787 0.66102993 0.16236761]
 [0.01103636 0.16599609 0.08242705 0.60649276 0.13404773]
 [0.5106585  0.2489046  0.00453031 0.21414617 0.02176036]
 [0.00007199 0.00028541 0.00016848 0.33291507 0.66655916]
 [0.1948554  0.11183021 0.69263786 0.00055823 0.00011824]
 [0.00294398 0.02972992 0.00072291 0.30623966 0.6603636 ]
 [0.00115515 0.00236157 0.0003819  0.7206616  0.27543986]
 [0.00816596 0.13927253 0.00046934 0.473

INFO:tensorflow:global_step/sec: 16.3438
INFO:tensorflow:probabilities = [[0.06032816 0.07914349 0.26098406 0.23299612 0.36654824]
 [0.00156578 0.00725921 0.00888386 0.21952423 0.76276696]
 [0.00291334 0.05300454 0.01729468 0.43709442 0.48969302]
 [0.00291427 0.02407819 0.0038497  0.7732999  0.19585793]
 [0.00110206 0.01569472 0.00073583 0.48012692 0.50234044]
 [0.00833215 0.34891725 0.02374807 0.36488873 0.25411373]
 [0.00259488 0.02026992 0.00165257 0.607058   0.3684247 ]
 [0.00124099 0.00209147 0.00002295 0.1636063  0.8330383 ]
 [0.00246811 0.09032438 0.04089286 0.4835971  0.3827175 ]
 [0.00645956 0.0518382  0.00092759 0.6315462  0.3092285 ]
 [0.00004035 0.00033763 0.00004924 0.05914965 0.94042313]
 [0.05959616 0.67197335 0.1544859  0.10466132 0.00928336]
 [0.05063026 0.14153475 0.00583379 0.55541104 0.24659023]
 [0.00164048 0.02102378 0.0167159  0.3958605  0.5647594 ]
 [0.00171218 0.08889241 0.00131618 0.5967695  0.31130978]
 [0.00467527 0.07543295 0.01703804 0.4283086  0.47454518]

INFO:tensorflow:loss = 0.77287257, step = 9901 (6.770 sec)
INFO:tensorflow:probabilities = [[0.01064989 0.27277163 0.68311894 0.02950567 0.00395384]
 [0.02117595 0.39395866 0.00059944 0.4871263  0.09713971]
 [0.02028163 0.1745734  0.5490325  0.1974407  0.05867175]
 [0.01907656 0.72109723 0.00130145 0.21610841 0.04241631]
 [0.02516909 0.4234806  0.00666241 0.5077456  0.03694231]
 [0.06203989 0.42049295 0.02885189 0.44969591 0.03891939]
 [0.14837857 0.02231365 0.00125544 0.47027862 0.35777378]
 [0.00120144 0.00884039 0.00024429 0.6005111  0.3892028 ]
 [0.00973402 0.00244397 0.00035676 0.30943263 0.67803264]
 [0.03775318 0.01379199 0.00105015 0.4847246  0.4626801 ]
 [0.9980702  0.00160451 0.00031955 0.00000532 0.00000038]
 [0.05315563 0.44488287 0.0062393  0.3475552  0.14816703]
 [0.01569215 0.12341502 0.426424   0.33328494 0.10118387]
 [0.00000616 0.00030805 0.00116639 0.40289822 0.59562117]
 [0.01105371 0.5356655  0.00756723 0.31940132 0.12631221]
 [0.03686991 0.2056195  0.1561729  0.45

In [54]:
style_classifier.evaluate(
        input_fn=get_validation_dataset,
        steps=1000)

INFO:tensorflow:Calling model_fn.
called
called
called
cnn_model_fn
Tensor("IteratorGetNext:0", shape=(?, 28, 28), dtype=float32)
Tensor("IteratorGetNext:1", shape=(?,), dtype=int64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-23T15:54:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpybklolrl/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Finished evaluation at 2019-03-23-15:54:45
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.7, global_step = 10000, loss = 0.73838174
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpybklolrl/model.ckpt-10000


{'accuracy': 0.7, 'global_step': 10000, 'loss': 0.73838174}